In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.jpmorgan.com/us/en/students/programs/data-analytics-opportunities")
page_data = loader.load().pop().page_content
print(page_data)







AI & Data Science Opportunities | JPMorganChase & Co. 



































We no longer support this browser.  Using a supported browser will provide a better experience.
Please update your browser.



Close browser message







Skip to Main Content









Open Navigation








Students programs






Work with us






Grow with us






How we hire






Careers















Submit to search 














                        Jobs, Student Programs & Internships 
                        













Data & Analytics
















Help us harness the power of data, analytics and insights


Our Data & Analytics teams deliver excellence at the intersection of data science, research and industry expertise. We develop technology and create solutions to help solve some of the world's most interesting financial problems, while improving our customer and client experiences every day. Whether you’re working with artificial intelligence, big data, machine le

In [14]:

from langchain_core.prompts import PromptTemplate
import json


prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your task is to extract job postings and return **only** a valid JSON object 
    with the following structure:
    {{
        "role": "Role name",
        "experience": "Experience required (if mentioned, else 'Not specified')",
        "skills": ["skill1", "skill2", ...],
        "description": "Brief description about the role"
    }}
    Return only valid JSON with no extra text, preamble, or explanation.
    ### VALID JSON:
    """
)

In [15]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke({'page_data': page_data})

In [16]:
content = res.content.strip()
if content.startswith("```json"):
    content = content[7:]
if content.endswith("```"):
    content = content[:-3]

In [18]:
# 6. Parse JSON
parsed_res = json.loads(content)

# 7. Print or use parsed result
print(json.dumps(parsed_res, indent=2))

{
  "role": "Data & Analytics",
  "experience": "Not specified",
  "skills": [
    "Machine learning",
    "Data science",
    "Python",
    "Matlab",
    "C++",
    "Java",
    "C#",
    "Big data analytics",
    "Deep learning",
    "Reinforcement learning",
    "Natural language processing"
  ],
  "description": "Help us harness the power of data, analytics and insights to develop technology and create solutions to help solve some of the world's most interesting financial problems"
}


In [19]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data & Analytics',
 'experience': 'Not specified',
 'skills': ['Machine learning',
  'Data science',
  'Python',
  'Matlab',
  'C++',
  'Java',
  'C#',
  'Big data analytics',
  'Deep learning',
  'Reinforcement learning',
  'Natural language processing'],
 'description': "Help us harness the power of data, analytics and insights to develop technology and create solutions to help solve some of the world's most interesting financial problems"}

In [20]:
type(json_res)

dict

In [38]:
import pandas as pd

df = pd.read_csv("D:\\coldemailgenerator\\app\\resources\\my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [36]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [31]:
job=json_res
job['skills']

['Machine learning',
 'Data science',
 'Python',
 'Matlab',
 'C++',
 'Java',
 'C#',
 'Big data analytics',
 'Deep learning',
 'Reinforcement learning',
 'Natural language processing']

In [32]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'li

In [33]:
job

{'role': 'Data & Analytics',
 'experience': 'Not specified',
 'skills': ['Machine learning',
  'Data science',
  'Python',
  'Matlab',
  'C++',
  'Java',
  'C#',
  'Big data analytics',
  'Deep learning',
  'Reinforcement learning',
  'Natural language processing'],
 'description': "Help us harness the power of data, analytics and insights to develop technology and create solutions to help solve some of the world's most interesting financial problems"}

In [34]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Data-Driven Solutions with AtliQ

Dear Hiring Manager,

I came across the Data & Analytics role at your organization and was impressed by the opportunity to harness the power of data, analytics, and insights to develop technology and create solutions to solve some of the world's most interesting financial problems. As a Business Development Executive at AtliQ, I'd like to introduce you to our company's capabilities in fulfilling your needs.

At AtliQ, we specialize in AI & Software Consulting, dedicated to facilitating the seamless integration of business processes through automated tools. Our expertise in machine learning, data science, and big data analytics can help you develop tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our portfolio showcases our expertise in machine learning and Python, with notable projects featured in our [ML-Python portfolio](https://example.com/ml-python-portfolio). W